# Debug LLM Kernel in VS Code

This notebook helps debug the LLM Kernel using VS Code's debugger.

## Option 1: Debug with Environment Variables

1. Set environment variables before starting VS Code:
```bash
export LLM_KERNEL_DEBUGGER=true
export LLM_KERNEL_DEBUG_WAIT=true
code .
```

2. Open this notebook and select LLM Kernel
3. Go to Run and Debug → "Debug LLM Kernel"
4. Run the cell below - kernel will wait for debugger

In [ ]:
# This will pause until debugger attaches
print("If kernel is waiting, attach debugger now!")

## Option 2: Enable Debugging Dynamically

In [1]:
# Enable debugging for next cell
import os
os.environ['LLM_KERNEL_DEBUGGER'] = 'true'

# Get kernel instance
kernel = get_ipython().kernel

# Start debugger
try:
    import debugpy
    debugpy.listen(5678)
    print("Debugger listening on port 5678")
    print("Go to VS Code → Run and Debug → 'Debug LLM Kernel'")
except ImportError:
    print("Install debugpy: pip install debugpy")

Debugger listening on port 5678
Go to VS Code → Run and Debug → 'Debug LLM Kernel'


## Option 3: Debug Specific Methods

In [ ]:
# Add breakpoint to specific method
kernel = get_ipython().kernel

# Wrap method with debugger
original_query = kernel.query_llm_sync

def debug_query(*args, **kwargs):
    import debugpy
    debugpy.breakpoint()  # Stops here!
    return original_query(*args, **kwargs)

kernel.query_llm_sync = debug_query
print("Breakpoint added to query_llm_sync")

In [ ]:
# This will hit the breakpoint
%%llm
Hello! This query will stop at the breakpoint.

## Inspect Kernel State

In [1]:
# Inspect kernel state
kernel = get_ipython().kernel

print(f"Kernel type: {type(kernel)}")
print(f"Active model: {kernel.active_model}")
print(f"Display mode: {kernel.display_mode}")
print(f"Context strategy: {kernel.context_manager.context_strategy}")
print(f"Execution count: {kernel.execution_count}")
print(f"\nPinned cells: {kernel.execution_tracker.pinned_cells}")
print(f"Cell mappings: {kernel.execution_tracker.cell_number_to_id}")

Kernel type: <class 'llm_kernel.kernel.LLMKernel'>
Active model: gpt-4o-mini
Display mode: inline
Context strategy: smart
Execution count: 1

Pinned cells: set()
Cell mappings: {1: '6de1fd03-fd1f-4ff6-8baa-3ad5e68dcecc'}


## Debug Context Issues

In [2]:
# Debug context tracking
kernel = get_ipython().kernel

# Set breakpoint in context manager
import debugpy

original_get_context = kernel.context_manager.get_context_for_model

def debug_get_context(*args, **kwargs):
    debugpy.breakpoint()
    return original_get_context(*args, **kwargs)

kernel.context_manager.get_context_for_model = debug_get_context
print("Breakpoint added to context retrieval")

Breakpoint added to context retrieval


## Debug Async Issues

In [3]:
# Check async environment
import asyncio

try:
    loop = asyncio.get_event_loop()
    print(f"Event loop: {loop}")
    print(f"Is running: {loop.is_running()}")
except RuntimeError as e:
    print(f"No event loop: {e}")

# Check nest_asyncio
try:
    import nest_asyncio
    print("\nnest_asyncio available")
    print(f"Applied: {getattr(nest_asyncio, '_patched', False)}")
except ImportError:
    print("\nnest_asyncio not installed")

Event loop: <_WindowsSelectorEventLoop running=True closed=False debug=False>
Is running: True

nest_asyncio available
Applied: False


In [ ]:
%llm_debug